In [11]:
from sklearn.datasets import load_boston

boston = load_boston()
X = boston.data
y = boston.target

NUM_EPOCHS = 200
NUM_HIDDEN = 500
LR = 0.01

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def report(y_pred, y_true):
  rmse = np.sqrt(mean_squared_error(y_pred, y_true))
  r2 = r2_score(y_pred, y_true)
  print(f'RMSE: {rmse}\n R^2: {r2}')

In [13]:
reg = MLPRegressor(hidden_layer_sizes=[NUM_HIDDEN], max_iter=NUM_EPOCHS, learning_rate_init=LR)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

report(y_pred, y_test)

RMSE: 2.7094625459619923
 R^2: 0.9078503784321894


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        layers.Dense(13, activation="relu"),
        layers.Dense(NUM_HIDDEN, activation="relu"),
        layers.Dense(1),
    ]
)

optimizer = keras.optimizers.Adam(LR)
model.compile(loss='mse', optimizer=optimizer)

model.fit(X_train, y_train, epochs=NUM_EPOCHS, verbose=0)
y_pred = model.predict(X_test)

report(y_pred, y_test)

RMSE: 2.886279067832969
 R^2: 0.8967490938606175


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, NUM_HIDDEN)
        self.fc2 = nn.Linear(NUM_HIDDEN, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Training on', device)
model = Net().to(device)

X_train_pt = torch.FloatTensor(X_train).to(device)
X_test_pt = torch.FloatTensor(X_test).to(device)
y_train_pt = torch.FloatTensor(y_train).reshape(-1, 1).to(device)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for t in range(NUM_EPOCHS):
    optimizer.zero_grad()
    y_pred_pt = model(X_train_pt)
    loss = loss_function(y_pred_pt, y_train_pt)
    loss.backward()   
    optimizer.step()

    if t % 200 == 99:
        y_pred_pt = model(X_test_pt)
        print(t, loss.cpu().item())
        model.zero_grad()

y_pred = model(X_test_pt).reshape(-1).detach().cpu().numpy()
report(y_pred, y_test)

Training on cpu
99 9.828548431396484
RMSE: 2.9182788392335848
 R^2: 0.8841307772160005


In [16]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.025)
reg.fit(X_train, y_train)
y_pred_gbr = reg.predict(X_test)
report(y_pred_gbr, y_test)

RMSE: 2.622596775657215
 R^2: 0.9032026977927723


In [17]:
y_pred_ensemble = (y_pred_gbr + y_pred) / 2
report(y_pred_ensemble, y_test)

RMSE: 2.5077026427016684
 R^2: 0.91130189606792


In [18]:
import pandas as pd
pd.DataFrame({
    'ensemble': y_pred_ensemble,
    'y_test': y_test
})  

ensemble  y_test
0    33.889555    33.2
1     8.868236     8.1
2    33.458287    28.5
3    15.423956    14.6
4    19.343853    22.6
..         ...     ...
122  18.541550    16.2
123  14.053720    12.7
124  33.094287    33.1
125  33.629501    30.1
126  21.571927    21.5

[127 rows x 2 columns]